In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import os
import numpy as np
import pandas as pd
from urllib.parse import quote_plus          
import time
from urllib.request import (urlopen, urlparse, urlunparse, urlretrieve)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import math
import re
from selenium.webdriver.chrome.service import Service
import os 
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import pdb
import os
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import traceback         
from selenium.webdriver.common.proxy import Proxy, ProxyType
import csv
import requests
import json
import random
import psutil
from datetime import datetime
import pickle
import warnings
from bs4 import BeautifulSoup

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-setuid-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument("disable-infobars")
#chrome_options.add_argument("headless")
#chrome_options.add_argument("--start-maximized")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
os.environ['WDM_LOG_LEVEL'] = '0'
os.environ['WDM_LOG'] = "false"
warnings.filterwarnings("ignore")
chrome_path = 'C:\\Users\\cafel\\chromedriver.exe'

In [3]:
def selenium_scroll_down(driver):
    SCROLL_PAUSE_SEC = 1
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        new_height = driver.execute_script("return document.body.scrollHeight")
  
        if new_height == last_height:
            return 1
        last_height = new_height


def get_driver(chrome_path, chrome_options, url):
    driver = None
    count = 0
    
    while (driver == None) and (count < 10):
            try:
                driver = webdriver.Chrome(chrome_path, options=chrome_options)
            except Exception:
                count = count + 1
                clean_up()
                if driver: driver.quit()
                continue
    connect = False
    
    while connect == False: 
        try:
            driver.get(url)
            driver.implicitly_wait(10)
            connect = True
        except Exception:
            del driver
            driver = webdriver.Chrome(chrome_path, options=chrome_options)
            continue
    return driver

def reset_driver(driver, chrome_path, chrome_options, url, cookies):

    try :
        driver.quit()
        driver = get_driver(chrome_path, chrome_options, url, cookies)
    except Exception:
        driver = get_driver(chrome_path, chrome_options, url, cookies)
    return driver


def write_data(write_file, datas):

    for data in datas:
        write_file = write_file.append(data, ignore_index = True)
    
    return write_file


def get_list(chrome_path, chrome_options):
    for i in range(6, 24):
        base_url = 'https://home.treasury.gov/news/press-releases?title=remark&publication-start-date=20{:02d}-01-01&publication-end-date=20{:02d}-12-31'.format(i, i)
        try:
            driver = get_driver(chrome_path, chrome_options, base_url)
        except Exception:
            reset_driver(driver, chrome_path, chrome_options, base_url)

        dataframe = pd.DataFrame(columns = ['title', 'url'])
        count = 0
        page_exist = True

        while page_exist:
            try:
                datas = []
                selenium_scroll_down(driver)
                main = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CLASS_NAME, 'region.region-content'))).find_element(By.CLASS_NAME, 'content--2col__body')
                articles = main.find_elements(By.CSS_SELECTOR, 'div')

                for article in tqdm(articles):
                    data = {}
                    try:
                        title = article.find_element(By.CSS_SELECTOR, 'h3 > a').get_attribute('href')
                        url = article.find_element(By.CSS_SELECTOR, 'h3 > a').text
                    except Exception:
                        title = None
                        url = None

                    data['title'] = title
                    data['url'] = url
                    datas.append(data)

                dataframe = write_data(dataframe, datas)   

                count = count + 1
                driver.implicitly_wait(1)

                try:
                    next_btn = driver.find_element(By.XPATH, '//*[@id="block-hamilton-content"]/div/div/div[2]/nav/ul/li[7]/a/span[2]/img')
                    next_btn.click()
                except NoSuchElementException:
                    try:
                        next_btn = driver.find_element(By.XPATH, '//*[@id="block-hamilton-content"]/div/div/div[2]/nav/ul/li[6]/a/span[2]/img')
                        next_btn.click()
                    except NoSuchElementException:
                        try:
                            next_btn = driver.find_element(By.XPATH, '//*[@id="block-hamilton-content"]/div/div/div[2]/nav/ul/li[5]/a/span[2]/img')
                            next_btn.click()
                        except NoSuchElementException:
                            print("No more next buttons found.")
                            page_exist = False

            except Exception:
                print(count)
                break

        dataframe = dataframe.drop_duplicates(subset=['url'], keep = 'first')
        dataframe.to_csv('C:/Users/cafel/Desktop/USDT/USDT_article_url/USDT_article_url_20{:02d}.csv'.format(i), encoding = 'utf-8-sig', index = False)
        driver.quit()
        print("End")

In [4]:
get_list(chrome_path, chrome_options)

UnboundLocalError: local variable 'driver' referenced before assignment

In [7]:
url_list = pd.DataFrame()
for i in range(6, 24):
    temp = pd.read_csv('C:/Users/cafel/Desktop/USDT/USDT_article_url/USDT_article_url_20{:02d}.csv'.format(i), encoding='utf-8-sig')
    url_list = pd.concat([url_list, temp], ignore_index=True)

# Print or use url_list as needed
print(url_list)

                                                 title  \
0    https://home.treasury.gov/news/press-releases/...   
1    https://home.treasury.gov/news/press-releases/...   
2    https://home.treasury.gov/news/press-releases/...   
3    https://home.treasury.gov/news/press-releases/...   
4    https://home.treasury.gov/news/press-releases/...   
..                                                 ...   
976  https://home.treasury.gov/news/press-releases/...   
977  https://home.treasury.gov/news/press-releases/...   
978  https://home.treasury.gov/news/press-releases/...   
979  https://home.treasury.gov/news/press-releases/...   
980  https://home.treasury.gov/news/press-releases/...   

                                                   url  
0    Under Secretary for International Affairs Davi...  
1    Deputy Assistant Secretary Sobel Remarks on th...  
2    Interim Assistant Secretary for Financial Stab...  
3    Opening Remarks by Secretary Henry M. Paulson,...  
4    Secretary Hen

In [8]:
def get_review(chrome_path, chrome_options, url_list):
    for i in tqdm(range(656, len(url_list))):
        base_url = url_list['title'][i]
        try:
            driver = get_driver(chrome_path, chrome_options, base_url)
        except Exception:
            reset_driver(driver, chrome_path, chrome_options, base_url)
        
        action = ActionChains(driver)
        
        dataframe = pd.DataFrame(columns = ['title', 'date', 'content'])
        datas = []

        try:
            selenium_scroll_down(driver)
            main = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CLASS_NAME, 'clearfix.content__container'))).find_element(By.CLASS_NAME, 'region.region-content')
            title = main.find_element(By.CSS_SELECTOR, '#block-hamilton-page-title > h2 > span').text
            date = main.find_element(By.CSS_SELECTOR, '#block-hamilton-content > article > div > div.date-format.field.field--name-field-news-publication-date.field--type-datetime.field--label-hidden.field__item > time').text.replace(' ','-').replace(',','')
            paragraphs = main.find_elements(By.CSS_SELECTOR, '#block-hamilton-content > article > div > div.clearfix.text-formatted.field.field--name-field-news-body.field--type-text-long.field--label-hidden.field__item > p')
            content = ''
            
            for paragraph in paragraphs:
                temp = paragraph.text
                content = content + ' ' + temp
            
        except Exception:
            title = None
            date = None
            content = None
            
        data = {'title': title, 'date': date, 'content': content}
        datas.append(data)
        driver.close()
        
        dataframe = write_data(dataframe, datas)
        dataframe.to_csv('C:/Users/cafel/Desktop/USDT/USDT_article/{}.csv'.format(i), encoding ='utf-8-sig', index=False)
    
    print("End")

In [9]:
get_review(chrome_path, chrome_options, url_list)

100%|████████████████████████████████████████████████████████████████████████████████| 325/325 [31:06<00:00,  5.74s/it]


End


In [27]:
article_list = pd.DataFrame()

for i in tqdm(range(len(url_list))):
    temp = pd.read_csv('C:/Users/cafel/Desktop/USDT/USDT_article/{}.csv'.format(i), encoding='utf-8-sig')
    
    # Replace multiple values in the 'content' column using re.sub
    temp['content'] = temp['content'].replace(['-30-', '###', 'Archived Content'], '', regex=True)
    
    # Apply re.sub only to non-null string values
    temp['content'] = temp['content'].apply(lambda x: re.sub("[()]", "", str(x)) if pd.notnull(x) else x)
    
    article_list = pd.concat([article_list, temp], ignore_index=True)

# Print or use article_list as needed
print(article_list)

100%|███████████████████████████████████████████████████████████████████████████████| 981/981 [00:01<00:00, 575.12it/s]


                                                 title              date  \
0    Under Secretary for International Affairs Davi...  December-16-2008   
1    Deputy Assistant Secretary Sobel Remarks on th...  December-10-2008   
2    Interim Assistant Secretary for Financial Stab...   December-5-2008   
3    Opening Remarks by Secretary Henry M. Paulson,...   December-3-2008   
4    Secretary Henry M. Paulson, Jr. Remarks on U.S...   December-2-2008   
..                                                 ...               ...   
976  Remarks by Chief Recovery Officer Jacob Leiben...   January-19-2023   
977  Remarks by Deputy Secretary of the Treasury Wa...   January-18-2023   
978  Remarks by Secretary of the Treasury Janet L. ...   January-18-2023   
979  Remarks by Deputy Secretary of the Treasury Wa...   January-17-2023   
980  Remarks by Secretary of the Treasury Janet L. ...   January-10-2023   

                                               content  
0      HP-1329 On U.S.-Asia Ec

In [28]:
article_list.to_csv('C:/Users/cafel/Desktop/USDT/USDT_article_total.csv', encoding ='utf-8-sig', index=False)